# K-Means Clustering Algorithm

### CSEC620 - Project 2 - Mehul Sen

K-means is a clustering algorithm used in machine learning and data analysis. It partitions a dataset into a specified number of clusters (K) by iteratively assigning data points to the nearest cluster centroid and updating the centroids based on the mean of the data points assigned to each cluster. The algorithm continues this process until convergence, resulting in clusters where data points within the same cluster are closer to each other than to data points in other clusters. K-means is unsupervised and seeks to find natural groupings or patterns within the data.

Two datasets are used in this assignment. The Iris and the Wine datasets. \
The Iris dataset which is a dataset containing information of Iris plants, it has three categories: Iris Setosa, Iris Versicolour, and Iris Virginica.
This dataset has the following four features:
- sepal length in cm
- sepal width in cm
- petal length in cm
- petal width in cm

The Wine dataset is a dataset containing Wine recognition data, it has three categories: 1, 2, and 3.
This dataset has the following thirteen features:
- Alcohol
- Malic acid
- Ash
- Alcalinity of ash
- Magnesium
- Total phenols
- Flavanoids
- Nonflavanoid phenols
- Proanthocyanins
- Color intensity
- Hue
- OD280/OD315 of diluted wines
- Proline   

### Import Required Libraries

In [1]:
# Import numpy for data handling, and basic math calculations
import numpy as np

### Dataset 1: Iris
Data points: 150 \
Features: 4 \
Clusters: 3

In [157]:
# Define the Dataset File
dataset = './Dataset/Iris/iris.data'
dataset_name = "Iris"
num_features = 0
clusters = 0

# Read dataset file
with open(dataset) as f:
    lines = f.readlines()
# Extract the number of features in the dataset from the first entry
num_features = len(lines[0].split(',')) - 1
# Initialize the 2d numpy array X
X = np.empty((len(lines), num_features))

# Extract values from the dataset into the numpy array X
for i, line in enumerate(lines):
  values = line.split(',')[:-1]  
  if len(values) == num_features:
    X[i] = np.array(values, dtype=float)

# Extract values from the dataset into the numpy array Y for labels 
Y = np.array([line.split(',')[-1].strip() for line in lines])
clusters = len(np.unique(Y))

print("Dataset: ", dataset_name)
print("\tNumber of Datapoints: ", X.shape[0])
print("\tNumber of Features: ", num_features)
print("\tNumber of Clusters: ", clusters)

Dataset:  Iris
	Number of Datapoints:  150
	Number of Features:  4
	Number of Clusters:  3


### Dataset 2: Wine
Data points: 178 \
Features: 13 \
Clusters: 3

In [104]:
# Define the Dataset File
dataset = './Dataset/wine/wine.data'
dataset_name = "Wine"
num_features = 0
clusters = 0

# Read dataset file
with open(dataset) as f:
    lines = f.readlines()
# Extract the number of features in the dataset from the first entry 
num_features = len(lines[0].split(',')[1:])
# Initialize the 2d numpy array X
X = np.empty((len(lines), num_features)) 

# Extract values from the dataset into the numpy array X
for i, line in enumerate(lines):
    values = line.split(',')[1:]
    if len(values) == num_features:
        X[i] = np.array(values, dtype=float)
        
# Extract labels Y from the first column
Y = np.array([line.split(',')[0] for line in lines])
clusters = len(np.unique(Y))

print("Dataset: ", dataset_name)
print("\tNumber of Datapoints: ", X.shape[0])
print("\tNumber of Features: ", num_features)
print("\tNumber of Clusters: ", clusters)

Dataset:  Wine
	Number of Datapoints:  178
	Number of Features:  13
	Number of Clusters:  3


### Helper functions
The following three helper functions are implemented below to evaluate the performance of the algorithms:
- **calculate_confusion_matrix**: It calculates the confusion matrix for a classification problem given the true labels and the predicted labels. The confusion matrix is a square matrix where each row represents the true class, and each column represents the predicted class. The values in the matrix count the number of samples that belong to each combination of true and predicted classes.
- **calculate_accuracy**:  It calculates the accuracy of a classification model by comparing the true labels with the predicted labels. The accuracy is the ratio of correctly predicted samples to the total number of samples, expressed as a percentage.
- **calculate_metrics_from_confusion_matrix**: It calculates three classification metrics, namely the precision, recall and f1 score.

In [163]:
# Define a function to calculate the confusion matrix
def calculate_confusion_matrix(true_labels, predicted_labels, num_classes):
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)
    for i in range(len(true_labels)):
        true_label = int(true_labels[i])
        predicted_label = int(predicted_labels[i])
        confusion_matrix[true_label, predicted_label] += 1
    return confusion_matrix

# Calculate the accuracy
def calculate_accuracy(true_labels, predicted_labels):
    correct_predictions = np.sum(true_labels == predicted_labels)
    total_samples = len(true_labels)
    accuracy = correct_predictions / total_samples
    return accuracy

def calculate_metrics_from_confusion_matrix(confusion_matrix):
    # Calculate Precision
    true_positives = confusion_matrix[clusters-1, clusters-1]
    false_positives = np.sum(confusion_matrix[:, clusters-1]) - true_positives
    if(true_positives + false_positives)!= 0:
        precision = true_positives / (true_positives + false_positives)
    else:
        precision = 0

    # Calculate Recall
    true_positives = confusion_matrix[clusters-1, clusters-1]
    false_negatives = np.sum(confusion_matrix[clusters-1, :]) - true_positives
    if(true_positives + false_positives)!= 0:
        recall = true_positives / (true_positives + false_negatives)
    else:
        recall = 0

    # Calculate F1 Score
    if(precision + recall)!= 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0
    return precision, recall, f1_score

### KMeans Algorithm Implementation
Implements the k-means algorithm as discussed in class.


In [160]:
class KMeans:
    def __init__(self, K=3, max_iters=100):
        # Number of clusters 
        self.K = K  
        # Maximum number of iterations for convergence
        self.max_iters = max_iters    
        # Lists to store cluster data
        self.clusters = [[] for _ in range(self.K)]  
        # Centroids for each cluster
        self.centroids = []

    def euclidean_distance(self, x1, x2):
        # Calculate Euclidean distance between two vectors
        return np.sqrt(np.sum((x1-x2)**2))

    def predict(self, X):
        # Store input data 
        self.X = X
        self.n_samples, self.n_features = X.shape    
        # Initialize centroids to random samples
        random_sample_idxs = np.random.choice(self.n_samples, self.K, replace=False)
        self.centroids = [self.X[idx] for idx in random_sample_idxs]
        # Optimize clusters  
        for _ in range(self.max_iters):    
            # Assign samples to closest centroids
            self.clusters = self.create_clusters(self.centroids)        
            # Calculate new centroids from assigned clusters
            centroids_old = self.centroids
            self.centroids = self.get_centroids(self.clusters)  
            # Check for convergence
            if self.is_converged(centroids_old, self.centroids):
                break
        # Return cluster labels        
        return self.get_cluster_labels(self.clusters)

    # Assign cluster labels to each sample
    def get_cluster_labels(self, clusters):
        labels = np.empty(self.n_samples)
        for cluster_idx, cluster in enumerate(clusters):
            for sample_idx in cluster:
                labels[sample_idx] = cluster_idx
        return labels

    # Assign samples to closest cluster by centroid   
    def create_clusters(self, centroids):
        clusters = [[] for _ in range(self.K)]
        for idx, sample in enumerate(self.X):
            # Get closest centroid index
            centroid_idx = self.closest_centroid(sample, centroids) 
            # Add sample index to cluster
            clusters[centroid_idx].append(idx)
        return clusters

    # Find closest centroid to a sample
    def closest_centroid(self, sample, centroids):
        # Distance of sample to each centroid
        distances = [self.euclidean_distance(sample, centroid) for centroid in centroids]  
        # Index of closest centroid
        closest_idx = np.argmin(distances)
        return closest_idx

    # Update centroids as cluster means
    def get_centroids(self, clusters):
        for cluster_idx, cluster in enumerate(clusters):
            # Calculate mean of cluster samples
            cluster_mean = np.mean(self.X[cluster], axis=0)
            # Update centroid
            self.centroids[cluster_idx] = cluster_mean
        return self.centroids
  
    # Check convergence between iteration
    def is_converged(self, centroids_old, centroids):
        # Distances between old and new centroids
        distances = [self.euclidean_distance(centroids_old[i], centroids[i]) for i in range(self.K)]
        # If sums to 0, there was no change
        return sum(distances) == 0

In [165]:
# Testing the KMeans Algorithm implementation
k = KMeans(K=clusters, max_iters=500)

# Run the prediction for number of iterations, and calculate the average for confusion, precision, recall, f1, and accuracy
num_iterations = 1000

# Initialize arrays to accumulate results
avg_confusion = np.zeros((clusters, clusters))
avg_precision = 0.0
avg_recall = 0.0
avg_f1_score = 0.0
avg_accuracy = 0.0

for _ in range(0, num_iterations):
    # Run the prediction
    y_pred = k.predict(X)

    # Create a mapping from string labels to unique integers
    label_to_int = {label: idx for idx, label in enumerate(np.unique(Y))}

    # Map true labels to integers
    true_labels_int = np.array([label_to_int[label] for label in Y])

    # Calculate the confusion matrix
    conf = calculate_confusion_matrix(true_labels_int, y_pred, clusters)   
   
    # Calculate precision, recall, and f1-score
    prec, rec, f1 =  calculate_metrics_from_confusion_matrix(conf)

    # Calculate accuracy
    acc = calculate_accuracy(true_labels_int, y_pred)

    # Accumulate results
    avg_confusion += conf
    avg_precision += prec
    avg_recall += rec
    avg_f1_score += f1
    avg_accuracy += acc

# Calculate averages
avg_confusion /= num_iterations
avg_precision /= num_iterations
avg_recall /= num_iterations
avg_f1_score /= num_iterations
avg_accuracy /= num_iterations

print("Perfomance: ")
print("\tConfusion: \n")
print(avg_confusion)
print("\tPrecision: ", avg_precision)
print("\tRecall: ", avg_recall)
print("\tF1 Score: ", avg_f1_score)
print("\tAccuracy: ", avg_accuracy)

Perfomance: 
	Confusion: 

[[16.485 17.49  16.025]
 [16.936 16.64  16.424]
 [15.996 17.017 16.987]]
	Precision:  0.3640076800486884
	Recall:  0.3397400000000003
	F1 Score:  0.31746469904291436
	Accuracy:  0.33407999999999916


### KMeans Mahalanobis Algorithm Implementation
Implements the k-means mahanalobis algorithm where the distance between euclideans is measured not using euclidean distance, but through mahanalobis. \
The formula for mahalanobis distance is as follows:
𝑑(𝑥, 𝑦) = (𝑥 - 𝑦)ᵀ𝐶(𝑥 - 𝑦)

In [166]:
class KMeans_Mahalanobis:
    def __init__(self, K=3, C=np.diag, max_iters=100):
        # Number of clusters 
        self.K = K
        # Mahalanobis Value
        self.C = C
        # Maximum number of iterations for convergence
        self.max_iters = max_iters
        # Lists to store cluster data
        self.clusters = [[] for _ in range(self.K)]
        # Centroids for each cluster
        self.centroids = []

    def mahalanobis_distance(self, x1, x2):
        # Calculate Mahalanobis distance between two vectors
        return np.sqrt((x1 - x2).T.dot(self.C).dot(x1 - x2))

    def predict(self, X):
        # Store imput data
        self.X = X
        self.n_samples, self.n_features = X.shape
        # Initialize centroids to random samples
        random_sample_idxs = np.random.choice(self.n_samples, self.K, replace=False)
        self.centroids = [self.X[idx] for idx in random_sample_idxs]
        # Optimize clusters
        for _ in range(self.max_iters):
            # Assign samples to closest centroids
            self.clusters = self.create_clusters(self.centroids)
            # Calculate new centroids from assigned clusters
            centroids_old = self.centroids
            self.centroids = self.get_centroids(self.clusters)
            # Check for convergence
            if self.is_converged(centroids_old, self.centroids):
                break
        # Return cluster labels
        return self.get_cluster_labels(self.clusters)

    # Assign cluster labels to each sample
    def get_cluster_labels(self, clusters):
        # each sample will get the label of the cluster it was assigned to
        labels = np.empty(self.n_samples)
        for cluster_idx, cluster in enumerate(clusters):
            for sample_idx in cluster:
                labels[sample_idx] = cluster_idx
        return labels

    # Assign samples to closest cluster by centroid  
    def create_clusters(self, centroids):
        clusters = [[] for _ in range(self.K)]
        for idx, sample in enumerate(self.X):
            # Get closest centroid index
            centroid_idx = self.closest_centroid(sample, centroids)
            # Add sample index to cluster
            clusters[centroid_idx].append(idx)
        return clusters

    # Find closest centroid to a sample
    def closest_centroid(self, sample, centroids):
        # Distance of sample to each centroid
        distances = [self.mahalanobis_distance(sample, centroid) for centroid in centroids]
        # Index of closest centroid
        closest_idx = np.argmin(distances)
        return closest_idx

    # Update centroids as cluster means
    def get_centroids(self, clusters):
        #centroids = np.zeros((self.K, self.n_features))
        for cluster_idx, cluster in enumerate(clusters):
            # Calculate mean of cluster samples
            cluster_mean = np.mean(self.X[cluster], axis=0)
            # Update centroid
            self.centroids[cluster_idx] = cluster_mean
        return self.centroids

    # Check convergence between iteration
    def is_converged(self, centroids_old, centroids):
        # Distances between old and new centroids
        distances = [self.mahalanobis_distance(centroids_old[i], centroids[i]) for i in range(self.K)]
        # If sums to 0, there was no change
        return sum(distances) == 0

In [190]:
# Tuned C values
if(dataset_name == "Wine"):
    C = np.diag([100, 1, 1, 1, 100, 1, 1000, 100, 1, 1000, 10, 1000, 100])
elif(dataset_name == "Iris"):
    C = np.diag([10, 10, 100, 100])
else:
    # Create a basic C with [1]s if a new dataset is used.
    C = np.diag([10] * num_features)

k = KMeans_Mahalanobis(K=clusters, C=C, max_iters=150)


# Run the prediction for number of iterations, and calculate the average for confusion, precision, recall, f1, and accuracy
num_iterations = 1000

# Initialize arrays to accumulate results
avg_confusion = np.zeros((clusters, clusters))
avg_precision = 0.0
avg_recall = 0.0
avg_f1_score = 0.0
avg_accuracy = 0.0

for _ in range(0, num_iterations):
    # Run the prediction
    y_pred = k.predict(X)

    # Create a mapping from string labels to unique integers
    label_to_int = {label: idx for idx, label in enumerate(np.unique(Y))}

    # Map true labels to integers
    true_labels_int = np.array([label_to_int[label] for label in Y])

    # Calculate the confusion matrix
    conf = calculate_confusion_matrix(true_labels_int, y_pred, clusters)   
   
    # Calculate precision, recall, and f1-score
    prec, rec, f1 =  calculate_metrics_from_confusion_matrix(conf)

    # Calculate accuracy
    acc = calculate_accuracy(true_labels_int, y_pred)

    # Accumulate results
    avg_confusion += conf
    avg_precision += prec
    avg_recall += rec
    avg_f1_score += f1
    avg_accuracy += acc

# Calculate averages
avg_confusion /= num_iterations
avg_precision /= num_iterations
avg_recall /= num_iterations
avg_f1_score /= num_iterations
avg_accuracy /= num_iterations

print("Perfomance: ")
print("\tConfusion: \n")
print(avg_confusion)
print("\tPrecision: ", avg_precision)
print("\tRecall: ", avg_recall)
print("\tF1 Score: ", avg_f1_score)
print("\tAccuracy: ", avg_accuracy)

Perfomance: 
	Confusion: 

[[16.824 16.893 16.283]
 [15.868 17.134 16.998]
 [15.568 16.928 17.504]]
	Precision:  0.3453235944591975
	Recall:  0.35007999999999995
	F1 Score:  0.3213389987746983
	Accuracy:  0.34307999999999966
